In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scripts.locator import Locator

### accessibility score: 
- distance to airport 
- distance to train stations
- altitude?

In [2]:
df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
df_airports = pd.read_csv("../data/processed/filtered_airports.csv")
df_railway = pd.read_csv("../data/processed/filtered_trains.csv")

In [3]:
df_municipalities.head()

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun
0,Álava,Alegría-Dulantzi,2971,1531,1440,alegria-dulantzi,alava,561.6857,-2.513507,42.84149,1001
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.6910,-3.001022,43.05265,1002
2,Álava,Aramaio,1381,709,672,aramaio,alava,381.8797,-2.566000,43.05400,1003
3,Álava,Artziniega,1856,913,943,artziniega,alava,196.9808,-3.128209,43.12220,1004
4,Álava,Armiñón,247,127,120,arminon,alava,463.5815,-2.872574,42.72305,1006


In [4]:
df_airports.head()

,airport_id,airport_name,latitude,longitude
0,1,Aeropuerto de Jerez,36.743828,-6.062658
1,2,Aeropuerto de Vigo,42.225740,-8.630491
2,3,Aeropuerto de Badajoz,38.889845,-6.821121
3,4,Aeropuerto de Granada - Jaén F.G.L.,37.186826,-3.778613
4,5,Aeropuerto de Almería,36.844781,-2.371234


In [5]:
df_railway.sample(15)

,station_code,station_name,latitude,longitude,province_clean,municipality_name_clean
577,67004,Maria De Huerva,41.540098,-0.997467,zaragoza,maria de huerva
543,32002,El Carpio,41.215445,-5.114406,valladolid,carpio
271,73003,Les Borges Blanques,41.521722,0.877266,lleida,"borges blanques, les"
587,81206,Gallur,41.867181,-1.321274,zaragoza,gallur
472,72101,Roda De Mar,41.186436,1.455518,tarragona,roda de berà
25,56301,Fiñana,37.184708,-2.839647,almeria,finana
187,70302,Sigüenza,41.071329,-2.646456,guadalajara,siguenza
594,71206,Fabara,41.234438,0.143302,zaragoza,fabara
276,81102,Agoncillo,42.443595,-2.291623,"rioja, la",agoncillo
458,71304,Pradell,41.133329,0.858616,tarragona,pradell de la teixeta


In [ ]:
# Apply the function to each row in the towns DataFrame and create two new columns
df_municipalities[['closest_distance_airport', 'nearest_airport']] = Locator(df_municipalities, df_airports, 'airport_name').result


In [11]:
df_municipalities.query("municipality_name_clean == 'amurrio'")

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun,closest_distance_airport,nearest_airport,closest_distance_train,nearest_train_station
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.691,-3.001022,43.05265,1002,28.762394,Aeropuerto de Bilbao,30.756835,Nanclares-Langraiz


In [10]:


df_municipalities[['closest_distance_train', 'nearest_train_station']]  = Locator(df_municipalities, df_railway, 'station_name').result


In [12]:
df_municipalities

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun,closest_distance_airport,nearest_airport,closest_distance_train,nearest_train_station
0,Álava,Alegría-Dulantzi,2971,1531,1440,alegria-dulantzi,alava,561.68570,-2.513507,42.84149,1001,18.273217,Aeropuerto de Vitoria,0.819736,Alegria-Dulantzi
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.69100,-3.001022,43.05265,1002,28.762394,Aeropuerto de Bilbao,30.756835,Nanclares-Langraiz
2,Álava,Aramaio,1381,709,672,aramaio,alava,381.87970,-2.566000,43.05400,1003,23.639652,Aeropuerto de Vitoria,23.176277,Alegria-Dulantzi
3,Álava,Artziniega,1856,913,943,artziniega,alava,196.98080,-3.128209,43.12220,1004,26.840606,Aeropuerto de Bilbao,43.021998,Nanclares-Langraiz
4,Álava,Armiñón,247,127,120,arminon,alava,463.58150,-2.872574,42.72305,1006,20.846277,Aeropuerto de Vitoria,2.240750,Manzanos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7947,Zaragoza,Biel,170,104,66,biel,zaragoza,754.24450,-0.936588,42.38749,50901,61.667966,Aeropuerto de Huesca Pirineos,17.531651,Riglos
7948,Zaragoza,Marracos,85,43,42,marracos,zaragoza,404.73610,-0.776047,42.09059,50902,37.961861,Aeropuerto de Huesca Pirineos,12.495085,Valença Do Minho
7949,Zaragoza,Villamayor de Gállego,2854,1446,1408,villamayor de gallego,zaragoza,231.12340,-0.773315,41.68518,50903,23.955690,Aeropuerto de Zaragoza,9.820085,Villanueva De Gallego
7950,Ceuta,Ceuta,83229,41980,41249,ceuta,ceuta,13.47725,-5.316195,35.88829,51001,1.008709,Helipuerto de Ceuta,29.010931,Algeciras


In [13]:
df_municipalities.to_csv("../data/scored/accessibility.csv", index=False)